# Differential Geometry Computations

This notebook is based on the original notebook: https://nbviewer.jupyter.org/github/sagemanifolds/SageManifolds/blob/master/Notebooks/SM_basic_Schwarzschild.ipynb

In this notebook we will use the metric of the two-dimensional euclidean space in polar coordinates. The corresponding tools have been developed within the [SageManifolds](https://sagemanifolds.obspm.fr) project.

For a given metric $g_{\mu\nu}$ we can compute:
- The inverse metric: $g^{\mu\nu}$.
- Christoffel Symbols: ${\Gamma^{\lambda}}_{\mu\nu}= \frac{1}{2}g^{\lambda\sigma}\left(\partial_\mu g_{\sigma\nu}+\partial_\nu g_{\sigma\mu}-\partial_\sigma g_{\mu\nu} \right)$.
- Riemann tensor: ${R^\lambda}_{\mu\nu\sigma}=\partial_\nu {\Gamma^\lambda}_{\mu\sigma}-\partial_\sigma {\Gamma^\lambda}_{\mu\nu}+\Gamma^\eta_{\mu\sigma}\Gamma^\lambda_{\eta\nu}-\Gamma^\eta_{\mu\nu}\Gamma^\lambda_{\eta\sigma}$.
- Ricci tensor: $R_{\mu\nu} = {R^\lambda}_{\mu\lambda\nu}$.
- Scalar curvature: $R = g^{\mu\nu}R_{\mu\nu}$.
- Einstein tensor: $G_{\mu\nu} = R_{\mu\nu}-\frac{1}{2}g_{\mu\nu}R$.

Although the notebook is created for $d$ dimensions, it can be generalized to whatever $d$ dimensional non-Lorentzian metric.

To run it, you must start SageMath with the Jupyter interface, via the command `sage -n jupyter`

In [1]:
version() #Sage version
%display latex #To display LaTeX expressions in some outputs

## Differentiable manifold

We define our differentiable manifold. The method `Manifold()` must receive the following arguments: $d$ dimensions of the manifold, and name of the manifold.

In [2]:
d=2 #space dimensions
M = Manifold(d, 'M')
print(M)

2-dimensional differentiable manifold M


## List of coordinates

We must define our coordinates via the method `chart()` applied to the object `M` (our manifold). Note that the argument of `chart()` is a raw string (hence the prefix `r` in front of it), which 
defines the range of each coordinate, if different from $(-\infty, +\infty)$, as well as its LaTeX symbol, if different from the Python symbol to denote the coordinate. The Python variables for each coordinate are declared within the `<...>` operator on the left-hand side of the identity, `X` denoting the Python variable chosen for the coordinate chart.

$S^2$ in spherical coordinates are the following:
$$
    \theta\in (0,\pi), \quad \phi\in(0,2\pi).
$$

In [3]:
X.<th,ph> = M.chart(r"th:(0,pi):\theta ph:(0,2*pi):\phi")
X

Chart (M, (th, ph))

In [4]:
X[:]

(th, ph)

The coordinates follows the same indexing: $X^0=r$, $X^1=\theta$.

In [5]:
X[0],X[1]

(th, ph)

## Metric tensor $g_{\mu\nu}$.

If we want to introduce a constant parameter $m$ as a symbolic positive variable, it must be done via the function `var()`:

In [6]:
#m = var('m') #To uncomment delete #
#assume(m>0) #To uncomment delete #
r = var('r')
assume(r>0) #To uncomment delete #

The metric tensor of the manifold `M` is returned by the method `metric()`; we initialize its components in the chart `X`, which is the default (unique) chart on `M`:

In [7]:
g = M.metric('g')
g[0,0] = r^2
g[1,1] = r^2*sin(th)^2
g.display()

g = r^2 dth*dth + r^2*sin(th)^2 dph*dph

To display the metric as a matrix:

In [8]:
g[:]

[          r^2             0]
[            0 r^2*sin(th)^2]

In order to access to a the component of the metric with components $(\mu,\nu)$ we would write: `g[mu,nu]`. Where `mu` and `nu` are integer variables such that $\mu,\nu \in \{0,\dots,d\}$, where $\{r,\theta\}\equiv\{0,1\}$ for our case. Here, we display the component $g_{rr}$.

In [9]:
g[0,0]

r^2

The inverse metric can be computed via `g.inverse()`.

In [10]:
ginv=g.inverse(); ginv

Tensor field inv_g of type (2,0) on the 2-dimensional differentiable manifold M

In [11]:
ginv.display()

inv_g = r^(-2) d/dth*d/dth + 1/(r^2*sin(th)^2) d/dph*d/dph

In [12]:
ginv[:]

[           r^(-2)                 0]
[                0 1/(r^2*sin(th)^2)]

If we multiply both matrices, we should get the $d\times d$ identity matrix

In [13]:
delta = g['_{ab}']*ginv['^{bc}']

In [14]:
delta[:]

[1 0]
[0 1]

## Christoffel symbols ${\Gamma^{\lambda}}_{\mu\nu}$.

The Christoffel symbols of $g$ with respect to the given coordinates are
printed by the method `christoffel_symbols_display()` applied to the metric object `g`. By 
default, only the nonzero symbols and the nonredundant ones (taking into account the symmetry of the last two indices) are displayed. Type `g.christoffel_symbols_display?` to see all possible options.

In [15]:
g.christoffel_symbols_display()

Gam^th_ph,ph = -cos(th)*sin(th) 
Gam^ph_th,ph = cos(th)/sin(th)

Accessing to a Christoffel symbol specified by its indices (e.g. ${\Gamma^r}_{\theta\,\theta}$):

In [16]:
g.christoffel_symbols()[0,1,1]

-cos(th)*sin(th)

Checking the symmetry on the last two indices:

In [17]:
g.christoffel_symbols()[0,0,1] == g.christoffel_symbols()[0,1,0]

True

## Riemann curvature tensor

The Riemann curvature tensor is obtained by the method `riemann()`:

In [18]:
Riem = g.riemann()
print(Riem)

Tensor field Riem(g) of type (1,3) on the 2-dimensional differentiable manifold M


Displaying its nonredundant components:

In [19]:
Riem.display_comp(only_nonredundant=True) #If there is no elements displayed means that all of them are identically zero.

Riem(g)^th_ph,th,ph = sin(th)^2 
Riem(g)^ph_th,th,ph = -1

We can **lower and raise all the indices** of the components $R^\lambda_{\ \, \mu\nu\sigma}$ of the Riemann tensor, via the metric $g$ by the methods `down()` and `up()`.

In [20]:
Riemdown = Riem.down(g);
Riemup = Riem.up(g);

In order to raise and lower just one index, one should use the metric. That is, ${R^{\lambda\mu}}_{\nu\sigma}={R^{\lambda}}_{\alpha\nu\sigma} g^{\alpha\mu}$.

In [21]:
Rup2down2 = Riem['^a_{bcd}']*ginv['^{be}']
Rup2down2.display_comp(only_nonredundant=True)

X^th,ph_th,ph = r^(-2) 
X^ph,th_th,ph = -1/r^2

Notice that the $X$ tensor is the Riemann tensor.

## Ricci tensor

We know that the Ricci tensor is computed via the Riemann curvature tensor: $R_{\mu\nu}={R^\lambda}_{\mu\lambda\nu}$. However, Sage can give us directly the Ricci tensor from the metric $g$ with the method `g.ricci()`.

In [22]:
Ric = g.ricci()

In [23]:
Ric.display()

Ric(g) = dth*dth + sin(th)^2 dph*dph

In [24]:
Ric[:]

[        1         0]
[        0 sin(th)^2]

Let us check that the definition of the Ricci tensor via the contraction of the Riemann tensor and the one given by the Sage method `g.ricci()` coincides. 

In [25]:
Ric == Riem.down(g)['_{abcd}']*ginv['^{ac}']

True

## Calculating the Scalar Curvature

It is computed by the contraction of theinverse metric and the Ricci tensor, i.e., $R = g^{\mu\nu} R_{\mu\nu}$.

In [26]:
ScalarCurvature=ginv['^{ab}']*Ric['_ab']; ScalarCurvature.expr()

2/r^2

## Kretschmann scalar

The Kretschmann scalar is the "square" of the Riemann tensor defined by 
$$ K = R_{\lambda\mu\nu\sigma} R^{\lambda\mu\nu\sigma}$$
To compute it, we must first form the tensor fields whose components are $R_{\lambda\mu\nu\sigma}$ and 
$R^{\lambda\mu\nu\sigma}$. They are obtained by respectively lowering and raising the indices of the components $R^\lambda_{\ \, \mu\nu\sigma}$ of the Riemann tensor, via the metric $g$. These two operations are performed by the methods `down()` and `up()`. The contraction is performed by summation on repeated indices:

In [27]:
K = Riem.down(g)['_{abcd}'] * Riem.up(g)['^{abcd}']
K

Scalar field on the 2-dimensional differentiable manifold M

In [28]:
K.display()

M --> R
(th, ph) |--> 4/r^4

In [29]:
K.expr()

4/r^4

## Levi-Civita Connection

The Levi-Civita Connection $\nabla$ associated with the metric $g$.

In [30]:
nab = g.connection() ; print(nab)

Levi-Civita connection nabla_g associated with the Riemannian metric g on the 2-dimensional differentiable manifold M


We check the compatibility of the connection with the metric (that is, $\nabla_g g=0$).

In [31]:
nab(g).display()

nabla_g(g) = 0

In [32]:
#w = M.vector_field('w')

Compute the covariant derivative of the vector $w = (r,r\sin\theta)$, $\nabla_\nu w^\nu$.

In [33]:
#w[:] = [r,r*sin(th)]
#print(nab(w))

In [34]:
#DW = (nab(w)['^a_b']*delta['_a^b'])
#DW.expr()

Check that $\nabla_\nu w^\nu = \partial_\nu w^\nu + w^\gamma {\Gamma^{\nu}}_{\gamma \nu}$.

In [35]:
#sum([w[i].diff(i)+w[i]*sum([g.christoffel_symbols()[j,i,j] for j in M.irange()]) for i in M.irange()])    